<a href="https://colab.research.google.com/github/Cepera90/test/blob/main/test_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Open inc_points_c
TC = pd.read_csv("/content/drive/MyDrive/test/inc_points_c.csv")

In [ ]:
TC.columns

Index(['Unnamed: 0', 'IncomingTrackId', 'Speed', 'Latitude', 'Longitude',
       'PointDate', 'Height', 'Acceleration', 'Deceleration', 'TickTimestamp',
       'AccelerationXOriginal', 'AccelerationYOriginal',
       'AccelerationZOriginal', 'TrackOrigin', 'os_version'],
      dtype='object')

In [3]:
#Delete the numeric data inc point, IncomingTrackId and track origin
TC = TC.drop(['Unnamed: 0','os_version','TickTimestamp','PointDate', 'IncomingTrackId'], axis=1)

In [4]:
#Delete the Latitude, Longitude and Height columns for anonimized data
TC = TC.drop(['Latitude','Longitude','Height'], axis=1)

In [7]:
# 'OriginalDriver' and 'Taxi' - 'Driver'
TC = TC.replace({'TrackOrigin':{'OriginalDriver':'Driver', 'Taxi':'Driver'}})

In [8]:
TC.loc[:,'TrackOrigin'].unique()

array(['Driver', 'Passanger'], dtype=object)

In [9]:
#Prepare data
X = TC.loc[:,TC.columns != 'TrackOrigin']
y = TC.loc[:,'TrackOrigin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [10]:
# DecisionTreeClassifier model
tree = DecisionTreeClassifier(criterion='entropy', max_depth=26, min_samples_leaf=4, splitter="random", random_state=1)

In [11]:
# DecisionTreeClassifier model train
tree.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=26, min_samples_leaf=4,
                       random_state=1, splitter='random')

Execution time 14s

In [12]:
# DecisionTreeClassifier model prediction
DTC_pred = tree.predict(X_test)

In [13]:
# Accuracy score DecisionTreeClassifier
print(accuracy_score(DTC_pred, y_test))

0.8015703601390644


Accuracy 0.8015703601390644

In [14]:
# BaggingClassifier with tree model train
model = BaggingClassifier(DecisionTreeClassifier(splitter='random'), n_estimators=40, random_state=1)
model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(splitter='random'),
                  n_estimators=40, random_state=1)

Execution time 5m (n_estimators=40)

In [15]:
# BaggingClassifier with tree model prediction
BC_pred = model.predict(X_test)

In [16]:
# Accuracy score BaggingClassifier with tree
print(accuracy_score(BC_pred, y_test))

0.8475680737909692


Accuracy 0.8475680737909692(n_estimators=40)

In [17]:
# Search parametrs RandomForestClassifier with RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 50, num = 10)]
min_samples_split = [int(x) for x in np.linspace(start = 30, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 10, stop = 30, num = 10)]

param_dist = {'n_estimators': n_estimators,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rs = RandomizedSearchCV(RandomForestClassifier(), 
                        param_dist, 
                        n_iter = 4, 
                        cv = 2, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(X_train, y_train)
rs.best_params_

Fitting 2 folds for each of 4 candidates, totalling 8 fits


{'min_samples_leaf': 12, 'min_samples_split': 50, 'n_estimators': 34}

In [18]:
# RandomForestClassifier model prediction
RFCs_pred = rs.predict(X_test)

In [19]:
# Accuracy score RandomForestClassifier
print(accuracy_score(RFCs_pred, y_test))

0.8376392017139763


In [ ]:
# RandomForestClassifier model train
forest = RandomForestClassifier(n_estimators=20, n_jobs=-1, random_state=1)
forest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=40, n_jobs=-1, random_state=1)

Execution time 7m

In [ ]:
# RandomForestClassifier model prediction
RFC_pred = forest.predict(X_test)

In [ ]:
# Accuracy score RandomForestClassifier
print(accuracy_score(RFC_pred, y_test))

0.8254841485969304


Accuracy 0.8254841485969304

In [ ]:
# ExtraTreesClassifier model train
Eforest = ExtraTreesClassifier(n_estimators=40, n_jobs=-1, random_state=1)
Eforest.fit(X_train, y_train)

ExtraTreesClassifier(n_estimators=40, n_jobs=-1, random_state=1)

Execution time 3m

In [ ]:
# ExtraTreesClassifier model prediction
EFC_pred = Eforest.predict(X_test)

In [ ]:
# Accuracy score ExtraTreesClassifier
print(accuracy_score(EFC_pred, y_test))

0.824581642769882
